## How to use Simple Coder

### Basic Implementation
Let's start with a very simple implementation to demonstrate how you can generate an output file with some simple instructions.

We will create an ```agenda``` object to organize and store our instructions, then use that to run our ```SimpleCoder``` agent.

Each ```SimpleCoder``` instance takes one instruction item to run, and outputs one file. We will handle complex agendas, messaging, etc somewhere else. This will allow us to simplify our code and focus on refining the core functionality, which features recursive calling that will be much easier to understand and troubleshoot without the additional overhead.

In [1]:
from simple_coder import SimpleCoder

C_WORKING_DIR = "~/temp/simple_coder/"

# We organize our instructions in a dict. 
# Later, we can easily load this from a JSON file or a database.
agenda = {
    "requirements": "I need a script that gets and prints the current weather in New York",
    "output_file_name": "weather.sh",
    "working_dir": C_WORKING_DIR
}

# Create a coder agent using the agenda
coder = SimpleCoder(**agenda)

# Run the agent
await coder.run()

Epoch: 0


'weather.sh'

The basic properties to our agenda are:
- ```requirements``` - State what you are trying to do here
- ```output_file_name``` - This is where we will write the code output. If the file already exists, the contents will be loaded as our initial source code for refactoring. Otherwise it will be created.
- ```working_dir``` - This is the base path for our input and output files

The agent will iterate at least 2 times, or until the LLM thinks it is complete and issues a stop code. This allows the LLM to analyze it's own output and make changes, increasing the quality of our output.

#### Let's check the output.

In [ ]:
!cat {agenda["working_dir"] + agenda["output_file_name"]}

## Using references 
We have illustrated how we can create a new file or refactor an existing one. 

However, projects often require multiple files, some of which are interdependant. It is very cumbersome to add all this source code to a chat bot, with your refactoring instructions.

With SimpleCoder, we can do this by adding a new key to our agenda:

```"input_file_name_list": ["project_readme.md", "project_file.py"]```

This allows us to specify a list of file names to be read in as references for our agent.

Let's see how this works:

In [ ]:
from simple_coder import SimpleCoder

C_WORKING_DIR = "~/temp/simple_coder/"

# Create an agenda list
# Note that the second element takes the output of the first element as input via "input_file_name_list"
agenda = [
    {
        "requirements": "I need a python class, WebLoader that downloads a URL, extracts text, and saves it to a file",
        "output_file_name": "web_loader.py",
        "working_dir": C_WORKING_DIR
    },
    {
        "requirements": "I need a python class WebScraper that uses WebLoader from webloader.py to scrape google for news mentioning AI",
        "output_file_name": "web_scraper.py",
        # This is a list of files that will be loaded as input for the agent. The agent is instructed to use these for reference.
        "input_file_name_list": ["web_loader.py"],
        "working_dir": C_WORKING_DIR
    }
]

# Iterate over our agenda and run an agent for each item
for i, item in enumerate(agenda):
    print(f"Agenda item #{i}: {item['output_file_name']} - starting")

    # Create and run the agent
    coder = SimpleCoder(**item)
    print( await coder.run() )
    
    print(f"Agenda item {i} - finished\n\n")

If you try the given example and check the output files, you should see that the ```WebScraper``` class will include an import and implementation of the ```WebLoader``` class.

## Limitations 

- The agent iterates until it determines that the code meets the specified requirements, and then issue a stop signal. However it can produce the stop code prematurely and generate only a partial solution. We attempt to migtigate this with specific prompt instructions for when the stop code may be generated.

- Because the agent simply examines the code and does not actually run it to debug, it can generate some syntactical or logical errors.

- The agent may iterrate endlessly, thinking that the requirements are not properly implement, and will keep making small adjustments. To mitigate, we implemented a max call limit, which will take the latest version of code as the output.

- Additionally, and probably most obviously, is that it increases the number of call-outs to the LLM. This is an issue for latency, but in our case we are seeking to optimize accuracy.